"Write a Python script that scrapes product details, raw materials details (from the provided Excel file), and brand details from the website https://natrue.org/our-standard/natrue-certified-world/?database[tab]=products. Extract data for each product, including product name, description, certification status, and other available details. Also, extract raw materials and brand information. Store all details in a structured JSON format and then export this data into Google Sheets. Ensure the script handles pagination on the website."


In [ ]:
pip install requests beautifulsoup4 pandas gspread oauth2client


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# Fonction pour récupérer les détails des produits
def get_product_details():
    url = "https://natrue.org/our-standard/natrue-certified-world/?database[tab]=products"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraire les informations des produits, ici un exemple
    products = []
    for product in soup.find_all('div', class_='product-class'):  # Exemple de classe à ajuster
        name = product.find('h3').text
        description = product.find('p', class_='product-description').text
        certification = product.find('span', class_='certification-status').text
        products.append({"name": name, "description": description, "certification": certification})

    return products

# Fonction pour envoyer les données vers Google Sheets
def send_to_google_sheets(data, sheet_name="ProductData"):
    # Créer des credentials pour Google Sheets API
    scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('/content/Projet_stage.json', scope)
    client = gspread.authorize(creds)

    # Ouvrir la feuille de calcul
    # Ouvrir la feuille de calcul
    sheet = client.open('stage').sheet1  # Remarquez les guillemets autour de 'stage'

    # Convertir en dataframe et envoyer
    df = pd.DataFrame(data)
    for i, row in df.iterrows():
        sheet.append_row(row.values.tolist())

# Récupérer les détails des produits et envoyer à Google Sheets
products = get_product_details()
send_to_google_sheets(products)

# Sauvegarder en JSON
with open('/content/sample_data/data.json', 'w') as json_file:
    json.dump(products, json_file, indent=4)


Temps estimé : 6–8 heures (scraping + tests + intégration).

Tâche 2 : Générer un prompt GPT pour extraire les matières premières de l'INCI et les structurer

Write a Python script that extracts individual raw materials from an INCI list and structures them similar to the material page at https://www.commonshare.com/materials/alpaca. The script should extract relevant details such as material name, description, properties, standards, and countries associated with each material. Ensure that the standards and countries are correctly matched with the materials. The final structure should be a well-organized data format, such as JSON, with each material having its own page for documentation.


In [ ]:
pip install requests beautifulsoup4 google-auth google-auth-oauthlib google-auth-httplib2 gspread


"Act as a data engineer. Write a Python script to parse INCI strings (ex: 'Aqua, Glycerin, Cocos Nucifera Oil') into individual raw materials. For each material, create a Google Doc mimicking https://www.commonshare.com/materials/alpaca with:

1. Standardized name (ISO 16128 ou INCI).
2. CAS Number (via PubChem API).
3. Sourcing Countries (géolocalisation via Wikidata).
4. Certifications (COSMOS, Ecocert, etc.).
5. Structure JSON : {name: 'Cocos Nucifera Oil', type: 'Plant', origin: ['Philippines', 'Brazil'], certifications: ['COSMOS']}.

Validate standards and countries with official databases. Save outputs to Google Drive."

In [ ]:
pip install requests gspread google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pywikibot pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.5/718.5 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 kB 15.1 MB/s eta 0:00:00


In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

def parse_inci(inci_string):
    materials = inci_string.split(', ')
    for material in materials:
        # Recherche CAS via PubChem
        cas = requests.get(f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{material}/property/InChIKey/TXT")
        # Géolocalisation via Wikidata...
        # Création Google Doc
        doc_body = {
            'title': material,
            'body': {
                'content': [{
                    'paragraph': {
                        'elements': [{
                            'textRun': {'content': f"CAS: {cas}\nOrigin: {countries}"}
                        }]
                    }
                }]
            }
        }
        drive_service.documents().create(body=doc_body).execute()

In [ ]:
def parse_inci_string(inc_str):
    materials = [material.strip() for material in inc_str.split(',')]
    return materials


In [ ]:
import requests

def get_cas_number(material):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{material}/property/CAS/JSON'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        try:
            return data['PropertyTable']['Properties'][0]['CAS']
        except (KeyError, IndexError):
            return None
    return None


In [ ]:
import os
os.environ["PYWIKIBOT_NO_USER_CONFIG"] = "1"


In [ ]:
from googleapiclient.http import MediaFileUpload


good :16HoOyG6PzmZcf4a8OoWXvhVjsp62n9Tr

In [8]:
import requests
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.service_account import Credentials
import time

# 1. Configuration des scopes et credentials
SCOPES = [
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/documents"
]

def get_google_credentials(credential_path):
    return Credentials.from_service_account_file(credential_path, scopes=SCOPES)

# 2. Parsing de la chaîne INCI
def parse_inci_string(inci_str):
    return [material.strip() for material in inci_str.split(',')]

# 3. Récupération du numéro CAS depuis PubChem (avec gestion d'erreur)
def get_cas_number(material):
    url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{material}/property/CAS/JSON'
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.json()['PropertyTable']['Properties'][0]['CAS']
        return None
    except Exception as e:
        print(f"Erreur CAS pour {material}: {str(e)}")
        return None

# 4. Géolocalisation depuis Wikidata (requête SPARQL améliorée)
def get_geolocation_from_wikidata(material):
    query = f"""
    SELECT ?countryLabel WHERE {{
        ?item rdfs:label "{material}"@en.
        ?item wdt:P17 ?country.
        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    try:
        response = requests.get(
            "https://query.wikidata.org/sparql",
            params={'query': query, 'format': 'json'},
            headers={'User-Agent': 'Mozilla/5.0'},
            timeout=15
        )
        return list(set([result['countryLabel']['value'] for result in response.json()['results']['bindings']]))
    except Exception as e:
        print(f"Erreur Wikidata pour {material}: {str(e)}")
        return []

# 5. Création du document Google dans votre dossier personnel
def create_google_doc(drive_service, docs_service, material_data, folder_id):
    try:
        # Création du document dans le dossier spécifié
        file_metadata = {
            'name': f"Fiche - {material_data['name']}",
            'parents': [folder_id],
            'mimeType': 'application/vnd.google-apps.document'
        }
        doc = drive_service.files().create(body=file_metadata).execute()
        doc_id = doc['id']

        # Ajout du contenu
        requests = [{
            'insertText': {
                'location': {'index': 1},
                'text': (
                    f"Nom: {material_data['name']}\n"
                    f"CAS: {material_data['cas']}\n"
                    f"Origines: {', '.join(material_data['origins'])}\n"
                    f"Certifications: {', '.join(material_data['certifications'])}\n"
                )
            }
        }]
        docs_service.documents().batchUpdate(
            documentId=doc_id,
            body={'requests': requests}
        ).execute()

        print(f"✅ Document créé: https://docs.google.com/document/d/{doc_id}")
        return doc_id
    except HttpError as error:
        print(f"❌ Erreur Google API: {error}")
        return None

# 6. Processus principal
def process_inci(inci_str, credentials_path, folder_id):
    credentials = get_google_credentials(credentials_path)
    drive_service = build('drive', 'v3', credentials=credentials)
    docs_service = build('docs', 'v1', credentials=credentials)

    for material in parse_inci_string(inci_str):
        print(f"\n🔍 Traitement de: {material}")

        doc_id = create_google_doc(
            drive_service,
            docs_service,
            {
                'name': material,
                'cas': get_cas_number(material) or "Non trouvé",
                'origins': get_geolocation_from_wikidata(material),
                'certifications': ['COSMOS']  # À adapter avec votre logique
            },
            folder_id
        )
        time.sleep(1)  # Respect des quotas d'API

# 7. Exécution
if __name__ == "__main__":
    # Configuration requise
    INCI_EXAMPLE = "Aqua, Glycerin, Cocos Nucifera Oil"
    CREDENTIALS_PATH = "/content/projetstage1.json"  # À modifier
    FOLDER_ID = "16HoOyG6PzmZcf4a8OoWXvhVjsp62n9Tr"  # À modifier

    process_inci(INCI_EXAMPLE, CREDENTIALS_PATH, FOLDER_ID)


🔍 Traitement de: Aqua
✅ Document créé: https://docs.google.com/document/d/1JzOzLtS7S4ooaKqsXgy1suWcBBk_OzaaIe4LwJttjeo

🔍 Traitement de: Glycerin
✅ Document créé: https://docs.google.com/document/d/14bz5jkp-hhozQ8ILB1UgPD-ujiIJ5nz_n4nMJSqBTAA

🔍 Traitement de: Cocos Nucifera Oil
✅ Document créé: https://docs.google.com/document/d/1Z7cJvSlgMKEnVxTDKnqrW1lQfPi90BdjFqyPoOyohng


Task 3 (Extra Credit) : Trouver les Contacts Clés

Outils Low-Code :



1.   Apollo.io ou Hunter.io : Pour trouver des emails et postes.
2.   LinkedIn Sales Navigator : Pour identifier les profils.
3.   Zapier : Automatiser la sauvegarde dans Google Sheets.
4.   Phantombuster : Scraper LinkedIn automatiquement.





In [12]:
import pandas as pd
from google.oauth2.service_account import Credentials
import gspread
import re

# 1. Connexion à Google Sheets (version corrigée)
def connect_google_sheets(credentials_path, sheet_id):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    try:
        creds = Credentials.from_service_account_file(credentials_path, scopes=SCOPES)
        client = gspread.authorize(creds)
        return client.open_by_key(sheet_id)
    except Exception as e:
        print(f"Erreur de connexion: {str(e)}")
        return None

# 2. Extraction du nom de marque (version améliorée)
def extract_brand(name):
    try:
        # Cas 1 : Séparateur "T/A"
        if " T/A " in name:
            return name.split(" T/A ")[-1].split("(")[0].strip()

        # Cas 2 : Marque entre crochets ou avec ™
        brand_match = re.search(r'(\[.*?\]™)|\((.*?)\)', name)
        if brand_match:
            return brand_match.group(1) or brand_match.group(2)

        # Cas 3 : Détection automatique du fabricant
        patterns = [
            r'(.*?)\s+(GmbH|SAS|AG|Ltd|INC|LLC)',
            r'(.*?)\s+\d',
            r'^(.*?)\s+-\s+'
        ]

        for pattern in patterns:
            match = re.match(pattern, name)
            if match:
                return match.group(1).strip()

        return name.split()[0]
    except:
        return "N/A"

# 3. Nettoyage des noms (version sécurisée)
def clean_company_names(df):
    try:
        df = df.copy()
        df['cleaned_name'] = df['Brand Name'].str.strip().str.upper()

        # Suppression des entités légales et caractères spéciaux
        df['cleaned_name'] = df['cleaned_name'].str.replace(
            r'\b(INC|LLC|LTD|SA|SAS|GMBH|PTY|CORP|PLC)\b[,.]?$',
            '',
            regex=True
        ).str.strip()

        df['cleaned_name'] = df['cleaned_name'].apply(
            lambda x: re.sub(r'[^a-zA-Z0-9À-ÿ& ]', '', x) if isinstance(x, str) else x
        )

        return df.drop_duplicates('cleaned_name').dropna(subset=['cleaned_name'])
    except KeyError:
        print("Colonne 'Brand Name' manquante!")
        return df

# 4. Workflow principal (version corrigée)
def main():
    # Configuration
    CREDS_PATH = '/content/projetstage1.json'
    SHEET_ID = '1gPypXrLKXphNi03Y9cl9njUJ7XEPFU6jaIETt7O4NdA'
    SHEET_NAME = 'Worksheet'

    try:
        # Connexion
        sheet = connect_google_sheets(CREDS_PATH, SHEET_ID)
        if not sheet:
            return

        worksheet = sheet.worksheet(SHEET_NAME)
        data = worksheet.get_all_values()

        # Création du DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])

        # Extraction des marques
        df["Brand Name"] = df["Name"].apply(extract_brand)

        # Nettoyage
        cleaned_df = clean_company_names(df)

        # Export Apollo
        apollo_template = pd.DataFrame({
            'Company Name': cleaned_df['cleaned_name'],
            'Target Job Titles': 'Marketing Manager|Business Development Manager'
        })

        apollo_template.to_csv('apollo_upload.csv', index=False)
        print("Export réussi! Fichier: apollo_upload.csv")

    except Exception as e:
        print(f"Erreur principale: {str(e)}")

if __name__ == "__main__":
    main()

Export réussi! Fichier: apollo_upload.csv
